In [1]:
%use serialization

In [2]:
val str = """
{
  "firstName": "John",
  "lastName": "Smith",
  "isAlive": true,
  "age": 27,
  "address": {
    "streetAddress": "21 2nd Street",
    "city": "New York",
    "state": "NY",
    "postalCode": "10021-3100"
  },
  "phoneNumbers": [
    {
      "type": "home",
      "number": "212 555-1234"
    },
    {
      "type": "office",
      "number": "646 555-4567"
    }
  ],
  "children": [
    "Catherine",
    "Thomas",
    "Trevor"
  ],
  "spouse": null
}
"""



In [3]:
str


{
  "firstName": "John",
  "lastName": "Smith",
  "isAlive": true,
  "age": 27,
  "address": {
    "streetAddress": "21 2nd Street",
    "city": "New York",
    "state": "NY",
    "postalCode": "10021-3100"
  },
  "phoneNumbers": [
    {
      "type": "home",
      "number": "212 555-1234"
    },
    {
      "type": "office",
      "number": "646 555-4567"
    }
  ],
  "children": [
    "Catherine",
    "Thomas",
    "Trevor"
  ],
  "spouse": null
}


In [4]:
fun validateJson(jsonString: String): Boolean {
    return try {
        val element = Json.Default.parseToJsonElement(jsonString)
        !(element is JsonPrimitive)
                && ((runCatching { element.jsonObject.entries.count() > 0 }.getOrNull() == true ||
                runCatching { element.jsonArray.size > 0 }.getOrNull() == true))  // JSON is valid
    } catch (e: Exception) {
        false // Invalid JSON
    }
}

In [5]:
fun String.printJson() = HTML(
    """
<body>
  <style>
  json-viewer {
    /* Background, font and indentation */
    --background-color: #0000;
    --font-size: 1.5rem;
  }
  </style>
  <script src="https://unpkg.com/@alenaksu/json-viewer@2.0.0/dist/json-viewer.bundle.js"></script>
  <json-viewer id="json" data='${this.replace("\n", "")}'></json-viewer>
  <script>
        document.querySelector('#json').expandAll()
  </script>
</body>
"""
)

In [6]:
USE {
    addRenderer(object : RendererHandler {
        override val execution: ResultHandlerExecution
            get() = ResultHandlerExecution { _, result ->
                FieldValue(
                    (result.value as String).printJson(),
                    result.name
                )
            }

        override fun accepts(value: Any?): Boolean {
            return value is String && validateJson(value)
        }

        override fun replaceVariables(mapping: Map<String, String>): RendererHandler {
            return this
        }
    })
}

In [7]:
USE {
    dependencies { implementation("com.sealwu.jsontokotlin:library:3.7.4") }
}

In [8]:
class DeserializationResut(val src: String, val className: String) {
    override fun toString() = src
}

fun String.deserialize(name : String) = DeserializationResut(this, name)

In [9]:
import wu.seal.jsontokotlin.model.TargetJsonConverter
import wu.seal.jsontokotlin.library.*

USE {
    updateVariable<DeserializationResut>() { value, kProperty ->
            execute("""
            import kotlinx.serialization.decodeFromString
            ${
                JsonToKotlinBuilder().run {
                    setAnnotationLib(TargetJsonConverter.Serializable).build(value.src, value.className)
                }
                    .replace("Any?", "Nothing?")
                    .replace("<Any>", "<Nothing>")
                    .replace("class ${value.className} : ArrayList<${value.className}Item>()", "typealias ${value.className} = ArrayList<${value.className}Item>")
                    .replace("class ${value.className} : ArrayList<Nothing>()", "typealias ${value.className} = ArrayList<Nothing>")
            }
            Json.Default.decodeFromString<${value.className}>(""" + "\"\"\"" + value.src + "\"\"\"" + """)
        """).name
    }
}


In [10]:
val res = str.deserialize("Result")

In [11]:
import kotlin.reflect.KProperty

class DeserializedDelegate(val serialized: String) {
    operator fun getValue(thisRef: Any?, property: KProperty<*>): DeserializationResut {
        return serialized.deserialize(property.name.replaceFirstChar { it.uppercase() })
    }
}

fun deserealize(serialized: String) = DeserializedDelegate(serialized)


In [12]:
val res by deserealize(str)

The problem is found in one of the loaded libraries: check library converters (fields callbacks)
java.lang.ClassCastException: class Line_14_jupyter$DeserializedDelegate cannot be cast to class Line_9_jupyter$DeserializationResut (Line_14_jupyter$DeserializedDelegate is in unnamed module of loader org.jetbrains.kotlin.scripting.compiler.plugin.impl.CompiledScriptClassLoader @2e1e96f4; Line_9_jupyter$DeserializationResut is in unnamed module of loader org.jetbrains.kotlin.scripting.compiler.plugin.impl.CompiledScriptClassLoader @2d57528e)
org.jetbrains.kotlinx.jupyter.exceptions.ReplLibraryException: The problem is found in one of the loaded libraries: check library converters (fields callbacks)
	at org.jetbrains.kotlinx.jupyter.exceptions.CompositeReplExceptionKt.throwLibraryException(CompositeReplException.kt:50)
	at org.jetbrains.kotlinx.jupyter.codegen.FieldsProcessorImpl.process(FieldsProcessorImpl.kt:54)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$1.invo

In [13]:
import java.net.URLConnection
import java.io.BufferedReader
import java.io.InputStreamReader
import java.net.HttpURLConnection
import java.net.URL

val page = 0
val httpURLConnection = URL("https://api.github.com/users/belovrv").openConnection() as HttpURLConnection

httpURLConnection.requestMethod = "GET"
val reader = BufferedReader(InputStreamReader(httpURLConnection.inputStream))

val response = reader.readText()

reader.close()

val belov = response.deserialize("Belov")

In [14]:
belov.followersUrl

https://api.github.com/users/belovrv/followers

In [15]:

val ktor_version = "2.3.2"

USE {
    dependencies {
        implementation("io.ktor:ktor-client-apache5-jvm:$ktor_version")
    }
}

In [16]:
import io.ktor.client.*
import io.ktor.client.request.*
import io.ktor.client.statement.*
import kotlinx.coroutines.runBlocking

val client = HttpClient()
fun HttpClient.getObject(url: String, typeName: String) = runBlocking { this@getObject.get(url).bodyAsText().deserialize(typeName) }

In [17]:
Json.Default.encodeToString(belov)

<json-viewer id="json" data='{"login":"belovrv","id":7994178,"node_id":"MDQ6VXNlcjc5OTQxNzg=","avatar_url":"https://avatars.githubusercontent.com/u/7994178?v=4","gravatar_id":"","url":"https://api.github.com/users/belovrv","html_url":"https://github.com/belovrv","followers_url":"https://api.github.com/users/belovrv/followers","following_url":"https://api.github.com/users/belovrv/following{/other_user}","gists_url":"https://api.github.com/users/belovrv/gists{/gist_id}","starred_url":"https://api.github.com/users/belovrv/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/belovrv/subscriptions","organizations_url":"https://api.github.com/users/belovrv/orgs","repos_url":"https://api.github.com/users/belovrv/repos","events_url":"https://api.github.com/users/belovrv/events{/privacy}","received_events_url":"https://api.github.com/users/belovrv/received_events","type":"User","site_admin":false,"name":null,"company":null,"blog":"","location":null,"email":null,"hireable":null,"bio":null,"twitter_username":null,"public_repos":12,"public_gists":1,"followers":8,"following":0,"created_at":"2014-06-26T08:38:58Z","updated_at":"2023-07-09T14:15:27Z"}'>

In [18]:
val s = "[]".deserialize("Test")
s


[]

In [19]:
class Test : ArrayList<Any>()

In [20]:
runBlocking { client.get(belov.organizationsUrl).bodyAsText() }

[]

In [21]:
val langs = client.getObject(subs[1].languagesUrl, "Langs")

Line_29.jupyter.kts (1:30 - 34) Unresolved reference: subs

In [22]:
langs

Line_30.jupyter.kts (1:1 - 6) Unresolved reference: langs

In [23]:
Json.Default.encodeToString(subs[0])

Line_31.jupyter.kts (1:29 - 33) Unresolved reference: subs

In [24]:
followers.map { it.login }

Line_32.jupyter.kts (1:1 - 10) Unresolved reference: followers
Line_32.jupyter.kts (1:17 - 19) Unresolved reference: it

In [25]:
%use dataframe

In [26]:
repos.map { it.mirrorUrl }

Line_37.jupyter.kts (1:1 - 6) Unresolved reference: repos
Line_37.jupyter.kts (1:13 - 15) Unresolved reference: it

In [27]:
class A(val s: String?, val b: String)
class B(val s: String?, val b: String, val C: A)

In [28]:
val collection = listOf(B(null, "1", A(null, "1")), B(null, "2", A(null, "2")))

In [29]:
collection.toDataFrame(maxDepth = 1)

<iframe onload="o_resize_iframe_out_1()" style="width:100%;" class="result_container" id="iframe_out_1" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_1157627904"></table>

<p class="dataframe_description">DataFrame: rowsCount = 2, columnsCount = 3</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for 

In [30]:
class C (val strings : List<String>)
val c = listOf(C(listOf("a", "b", "c")))
c.toDataFrame()

<iframe onload="o_resize_iframe_out_2()" style="width:100%;" class="result_container" id="iframe_out_2" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_1157627905"></table>

<p class="dataframe_description">DataFrame: rowsCount = 1, columnsCount = 1</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for 

In [31]:
val c = listOf(listOf("a", "b", "c"), listOf("a", "b", "c", "d"))
DISPLAY(c.toDataFrame())
DISPLAY(c.toDataFrame(maxDepth = 1))

<iframe onload="o_resize_iframe_out_3()" style="width:100%;" class="result_container" id="iframe_out_3" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_1157627906"></table>

<p class="dataframe_description">DataFrame: rowsCount = 1, columnsCount = 3</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for 

<iframe onload="o_resize_iframe_out_4()" style="width:100%;" class="result_container" id="iframe_out_4" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_1157627907"></table>

<p class="dataframe_description">DataFrame: rowsCount = 2, columnsCount = 1</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for 

In [33]:
repos.map { it.topics }.toDataFrame(maxDepth = 1)

Line_43.jupyter.kts (1:1 - 6) Unresolved reference: repos
Line_43.jupyter.kts (1:13 - 15) Unresolved reference: it

In [34]:
JsonToKotlinBuilder().run {
                    setAnnotationLib(TargetJsonConverter.Serializable).build(response, "Res")
                }


import kotlinx.serialization.SerialName
import kotlinx.serialization.Serializable

@Serializable
data class Res(
    @SerialName("login")
    val login: String,
    @SerialName("id")
    val id: Int,
    @SerialName("node_id")
    val nodeId: String,
    @SerialName("avatar_url")
    val avatarUrl: String,
    @SerialName("gravatar_id")
    val gravatarId: String,
    @SerialName("url")
    val url: String,
    @SerialName("html_url")
    val htmlUrl: String,
    @SerialName("followers_url")
    val followersUrl: String,
    @SerialName("following_url")
    val followingUrl: String,
    @SerialName("gists_url")
    val gistsUrl: String,
    @SerialName("starred_url")
    val starredUrl: String,
    @SerialName("subscriptions_url")
    val subscriptionsUrl: String,
    @SerialName("organizations_url")
    val organizationsUrl: String,
    @SerialName("repos_url")
    val reposUrl: String,
    @SerialName("events_url")
    val eventsUrl: String,
    @SerialName("received_events_url")
  

In [35]:
import kotlinx.serialization.SerialName
import kotlinx.serialization.Serializable

@Serializable
class Res : ArrayList<ResItem>()

@Serializable
data class ResItem(
    @SerialName("login")
    val login: String,
    @SerialName("id")
    val id: Int,
    @SerialName("node_id")
    val nodeId: String,
    @SerialName("avatar_url")
    val avatarUrl: String,
    @SerialName("gravatar_id")
    val gravatarId: String,
    @SerialName("url")
    val url: String,
    @SerialName("html_url")
    val htmlUrl: String,
    @SerialName("followers_url")
    val followersUrl: String,
    @SerialName("following_url")
    val followingUrl: String,
    @SerialName("gists_url")
    val gistsUrl: String,
    @SerialName("starred_url")
    val starredUrl: String,
    @SerialName("subscriptions_url")
    val subscriptionsUrl: String,
    @SerialName("organizations_url")
    val organizationsUrl: String,
    @SerialName("repos_url")
    val reposUrl: String,
    @SerialName("events_url")
    val eventsUrl: String,
    @SerialName("received_events_url")
    val receivedEventsUrl: String,
    @SerialName("type")
    val type: String,
    @SerialName("site_admin")
    val siteAdmin: Boolean
)

In [36]:
Json.Default.decodeFromString<Res>(response)

Unexpected JSON token at offset 2: Encountered an unknown key 'login' at path: $
Use 'ignoreUnknownKeys = true' in 'Json {}' builder to ignore unknown keys.
JSON input: {"login":"belovrv","id":7994178,.....
kotlinx.serialization.json.internal.JsonDecodingException: Unexpected JSON token at offset 2: Encountered an unknown key 'login' at path: $
Use 'ignoreUnknownKeys = true' in 'Json {}' builder to ignore unknown keys.
JSON input: {"login":"belovrv","id":7994178,.....
	at kotlinx.serialization.json.internal.JsonExceptionsKt.JsonDecodingException(JsonExceptions.kt:24)
	at kotlinx.serialization.json.internal.JsonExceptionsKt.JsonDecodingException(JsonExceptions.kt:32)
	at kotlinx.serialization.json.internal.AbstractJsonLexer.fail(AbstractJsonLexer.kt:584)
	at kotlinx.serialization.json.internal.AbstractJsonLexer.failOnUnknownKey(AbstractJsonLexer.kt:579)
	at kotlinx.serialization.json.internal.StreamingJsonDecoder.handleUnknown(StreamingJsonDecoder.kt:251)
	at kotlinx.serialization.json.

In [37]:
response

<json-viewer id="json" data='{"login":"belovrv","id":7994178,"node_id":"MDQ6VXNlcjc5OTQxNzg=","avatar_url":"https://avatars.githubusercontent.com/u/7994178?v=4","gravatar_id":"","url":"https://api.github.com/users/belovrv","html_url":"https://github.com/belovrv","followers_url":"https://api.github.com/users/belovrv/followers","following_url":"https://api.github.com/users/belovrv/following{/other_user}","gists_url":"https://api.github.com/users/belovrv/gists{/gist_id}","starred_url":"https://api.github.com/users/belovrv/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/belovrv/subscriptions","organizations_url":"https://api.github.com/users/belovrv/orgs","repos_url":"https://api.github.com/users/belovrv/repos","events_url":"https://api.github.com/users/belovrv/events{/privacy}","received_events_url":"https://api.github.com/users/belovrv/received_events","type":"User","site_admin":false,"name":null,"company":null,"blog":"","location":null,"email":null,"hireable":null,"bio":null,"twitter_username":null,"public_repos":12,"public_gists":1,"followers":8,"following":0,"created_at":"2014-06-26T08:38:58Z","updated_at":"2023-07-09T14:15:27Z"}'>

In [39]:
val s = """
["A", "B", "C"]
""".deserialize("Test")

The problem is found in one of the loaded libraries: check library converters (fields callbacks)
org.jetbrains.kotlinx.jupyter.exceptions.ReplEvalRuntimeException: Serializer for class 'Test' is not found.
Please ensure that class is marked as '@Serializable' and that the serialization compiler plugin is applied.

org.jetbrains.kotlinx.jupyter.exceptions.ReplLibraryException: The problem is found in one of the loaded libraries: check library converters (fields callbacks)
	at org.jetbrains.kotlinx.jupyter.exceptions.CompositeReplExceptionKt.throwLibraryException(CompositeReplException.kt:50)
	at org.jetbrains.kotlinx.jupyter.codegen.FieldsProcessorImpl.process(FieldsProcessorImpl.kt:54)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$1.invoke(CellExecutorImpl.kt:91)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$1.invoke(CellExecutorImpl.kt:90)
	at org.jetbrains.kotlinx.jupyter.config.LoggingKt.catchAll(logging.kt:42)
	at org.jetbrains.kotl

In [40]:
val ktor_version = "2.3.2"

USE {
    dependencies {
        implementation("io.ktor:ktor-client-apache5-jvm:$ktor_version")
    }
}

In [41]:
import io.ktor.client.*
import io.ktor.client.request.*
import io.ktor.client.statement.*
import kotlinx.coroutines.runBlocking

val client = HttpClient()
fun HttpClient.getObject(url: String, typeName: String) =
    runBlocking { this@getObject.get(url).bodyAsText().deserialize(typeName) }

In [42]:
val belov = client.getObject("https://api.github.com/users/belovrv", "User")

In [47]:
Json.Default.encodeToString(belov)

<json-viewer id="json" data='{"login":"belovrv","id":7994178,"node_id":"MDQ6VXNlcjc5OTQxNzg=","avatar_url":"https://avatars.githubusercontent.com/u/7994178?v=4","gravatar_id":"","url":"https://api.github.com/users/belovrv","html_url":"https://github.com/belovrv","followers_url":"https://api.github.com/users/belovrv/followers","following_url":"https://api.github.com/users/belovrv/following{/other_user}","gists_url":"https://api.github.com/users/belovrv/gists{/gist_id}","starred_url":"https://api.github.com/users/belovrv/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/belovrv/subscriptions","organizations_url":"https://api.github.com/users/belovrv/orgs","repos_url":"https://api.github.com/users/belovrv/repos","events_url":"https://api.github.com/users/belovrv/events{/privacy}","received_events_url":"https://api.github.com/users/belovrv/received_events","type":"User","site_admin":false,"name":null,"company":null,"blog":"","location":null,"email":null,"hireable":null,"bio":null,"twitter_username":null,"public_repos":12,"public_gists":1,"followers":8,"following":0,"created_at":"2014-06-26T08:38:58Z","updated_at":"2023-07-09T14:15:27Z"}'>